# Build a Deepfake Detection System 

This notebook illustrates how to build a video deepfake detection system from scratch using [Towhee](https://towhee.io/). A deepfake detection system predict videos with probabilities being fake ones. This tutorial will use examples from [Deepfake Detection Challenge](https://www.kaggle.com/competitions/deepfake-detection-challenge).

Using the sample data of different videos, we will build a basic deepfake detection system within a few lines of code and check the performance using Towhee. At the end, we use [Gradio](https://gradio.app/) to create a showcase that can be played with.

## Preparation

### Install packages

Make sure you have installed required python packages:

| package |
| -- |
| towhee |
| towhee.models |
| dlib |
| facenet-pytorch |
| albumentations |
| timm |
| pytorch_toolbelt |
| tensorboardx |
| tqdm |
| gradio |

In [1]:
! python -m pip install -q towhee towhee.models dlib facenet-pytorch albumentations timm pytorch_toolbelt tensorboardx tqdm gradio

### Prepare data

This tutorial will use a small data extracted from test dataset of [Deepfake Detection Challenge](https://www.kaggle.com/competitions/deepfake-detection-challenge/overview). You can download the subset from [Github](https://github.com/towhee-io/examples/releases/download/data/deepfake_video.zip). This tutorial will just use 2 videos under `test` as example.

The data is organized as follows:
- **test:** 2 videos from Deepfake Detection Challenge test dataset

Let's take a quick look:

In [8]:
! curl -L https://github.com/towhee-io/examples/releases/download/data/deepfake_video.zip -O
! unzip -q -o deepfake_video.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 14.3M  100 14.3M    0     0  1592k      0  0:00:09  0:00:09 --:--:-- 3052k


## Build System

Now we are ready to build a deepfake detection system using sample data. We will use the [Combined efficientnet](https://arxiv.org/abs/2107.02612) model to predict the possibilites of being fake ones for input videos. With proper [Towhee operators](https://towhee.io/operators), you don't need to go through video preprocessing & model details.

In [27]:
from towhee import ops, pipe, DataCollection

p = (
    pipe.input('path')
    .map('path', 'scores', ops.towhee.deepfake())
    .output('scores')
)

DataCollection(p('./deepfake_video/test/aagfhgtpmv.mp4')).show()

scores
0.99004173


#### Pipeline Explanation

Here are some details for each line of the assemble pipeline:

- `towhee.deepfake()`: a Towhee operator applying pretrained models to predict the probabilite a video being a fake one. The higher the score, the higher the probability of it being a fake video.[learn more](https://towhee.io/towhee/deepfake)

## Release a Showcase

We've learnt how to build a deepfake detection system. Now it's time to add some interface and release a showcase. We use `deepfake_detection_function` to wrap the pipeline as a [Gradio](https://gradio.app/) application.

In [28]:
def deepfake_detection_function(path):
    return p(path).get_dict()['scores']

In [29]:
import gradio

interface = gradio.Interface(deepfake_detection_function, 
                             inputs = gradio.Video(source='upload'),
                             outputs = [gradio.Textbox(lines=1)]
                            )

interface.launch(inline=True, share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://0fb299f8-a2b2-4257.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


loading state dict /home/xuyu/.towhee/operators/towhee/deepfake/main/weights/final_999_DeepFakeClassifier_tf_efficientnet_b7_ns_0_23
loading state dict /home/xuyu/.towhee/operators/towhee/deepfake/main/weights/final_777_DeepFakeClassifier_tf_efficientnet_b7_ns_0_31


<img src='deepfake.jpg' alt='deepfake_detection_demo' width=700px/>